In [1]:
import numpy as np
import math

### define functions:

In [2]:
def entropy(data, num_short_blocks=10):
    eol = np.sum(np.square(data))
    win_len = len(data)
    sub_win_len = math.floor(win_len / num_short_blocks)

    if win_len != sub_win_len * num_short_blocks:
        data = data[0:sub_win_len * num_short_blocks]
    sub_wins = data.reshape(sub_win_len, num_short_blocks, order='F').copy()
    norm_sub_frame_energies = np.zeros((1, sub_wins.shape[1]))
    for i in range(sub_wins.shape[1]):
        norm_sub_frame_energies[0, i] = np.sum(np.square(sub_wins[:, i])) / (eol + np.spacing(1))
    energy_entropy = 0
    for i in range(norm_sub_frame_energies.shape[1]):
        energy_entropy -= norm_sub_frame_energies[0, i] * math.log(norm_sub_frame_energies[0, i] + np.spacing(1), 2)
    return energy_entropy

In [3]:
def dft(data, f_s = 4000, p=0):
    win_len = len(data)
    fft = np.abs(np.fft.fft(data)) / win_len
    if not p:
        fft = fft[0:math.ceil(win_len)]
        f_req = (f_s / 2) * np.arange(0, np.ceil(win_len / 2) + 1) / np.ceil(win_len / 2)
    else:
        fft = np.fft.fftshift(fft)
        if win_len % 2:
            f_req = np.arange(-(win_len - 1) / 2, (win_len - 1) / 2 + 1)
        else:
            f_req = np.arange(-win_len / 2, win_len / 2)
    fft_1 = np.abs(fft)/win_len
    fft_2 = fft_1[1:(round(win_len / 2) + 1)]
    fft_2 = 2*fft_2
    return fft_2, f_req

In [4]:
def spectral_rolloff(data, c=0.90):
    total_energy = np.sum(np.square(data))
    curr_energy = 0
    count_fft = 0
    fft_len = len(data)
    while curr_energy <= c * total_energy and count_fft <= fft_len:
        curr_energy += data[count_fft] ** 2
        count_fft += 1
    count_fft -= 1
    return (count_fft - 1) / fft_len

In [5]:
def spectral_centroid(data, f_s = 4000):
    fft_len = len(data)
    m = np.transpose((f_s / (2 * fft_len)) * np.arange(1, fft_len+1))
    data = data / np.max(data)
    c = np.sum(np.multiply(m, data)) / (np.sum(data) + np.spacing(1))
    k = np.sum(np.square(m - c) * data)
    l = (np.sum(data) + np.spacing(1))
    
    if k*l <0 :
        s = math.sqrt( -1* k / l ) / (f_s / 2)
        c = c / (f_s / 2)
        return c
    else:
        s = math.sqrt( k / l ) / (f_s / 2)
        c = c / (f_s / 2)
        return c

In [6]:
def spectral_spread(data, f_s=4000):
    fft_len = len(data)
    m = np.transpose((f_s / (2 * fft_len)) * np.arange(1, fft_len+1))
    data = data / np.max(data)
    c = np.sum(np.multiply(m, data)) / (np.sum(data) + np.spacing(1))
    k = np.sum(np.square(m - c) * data)
    l = (np.sum(data) + np.spacing(1))
    
    if k*l <0 :
        s = math.sqrt( -1* k / l ) / (f_s / 2)
        c = c / (f_s / 2)
        return c
    else:
        s = math.sqrt( k / l ) / (f_s / 2)
        c = c / (f_s / 2)
        return s

### define data

In [7]:
path = "/Users/ecem/Desktop/gyrocardiogram/"
diseased_s = np.load(path + "data/diseased-10sec-scg-s.npy", allow_pickle= True)
diseased_r = np.load(path + "data/diseased-10sec-scg-r.npy", allow_pickle= True)

print(diseased_s.shape)

(3, 1521, 2560)


## Feature Extraction:


### Entropy:

In [8]:
entropy_s = np.ndarray((3,diseased_s.shape[1]))
for i in range(diseased_s.shape[1]):
    entropy_s[0,i] = entropy(diseased_s[0,i], num_short_blocks = 10)
    entropy_s[1,i] = entropy(diseased_s[1,i], num_short_blocks = 10)
    entropy_s[2,i] = entropy(diseased_s[2,i], num_short_blocks = 10)

In [9]:
entropy_r = np.ndarray((3,diseased_r.shape[1]))
for i in range(diseased_r.shape[1]):
    entropy_r[0,i] = entropy(diseased_r[0,i], num_short_blocks = 10)
    entropy_r[1,i] = entropy(diseased_r[1,i], num_short_blocks = 10)
    entropy_r[2,i] = entropy(diseased_r[2,i], num_short_blocks = 10)

### Spectral Entropy:

In [10]:
entropy(dft(diseased_r[0][1])[0])

1.7407195136627154

In [11]:
spec_ent_s = np.ndarray((3,diseased_s.shape[1]))
for i in range(diseased_s.shape[1]):
    x = dft(diseased_s[0,i])[0]
    y = dft(diseased_s[1,i])[0]
    z = dft(diseased_s[2,i])[0]
    
    spec_ent_s[0,i] = entropy(x, num_short_blocks = 10)
    spec_ent_s[1,i] = entropy(y, num_short_blocks = 10)
    spec_ent_s[2,i] = entropy(z, num_short_blocks = 10)

In [12]:
spec_ent_r = np.ndarray((3,diseased_r.shape[1]))
for i in range(diseased_r.shape[1]):
    x = dft(diseased_r[0,i])[0]
    y = dft(diseased_r[1,i])[0]
    z = dft(diseased_r[2,i])[0]
    
    spec_ent_r[0,i] = entropy(x, num_short_blocks = 10)
    spec_ent_r[1,i] = entropy(y, num_short_blocks = 10)
    spec_ent_r[2,i] = entropy(z, num_short_blocks = 10)

### Spectral Rolloff

In [13]:
spec_roll_s = np.ndarray((3,diseased_s.shape[1]))
for i in range(diseased_s.shape[1]):
    x = dft(diseased_s[0,i])[0]
    y = dft(diseased_s[1,i])[0]
    z = dft(diseased_s[2,i])[0]
    
    spec_roll_s[0,i] = spectral_rolloff(x)
    spec_roll_s[1,i] = spectral_rolloff(y)
    spec_roll_s[2,i] = spectral_rolloff(z)

In [14]:
spec_roll_r = np.ndarray((3,diseased_r.shape[1]))
for i in range(diseased_s.shape[1]):
    x = dft(diseased_r[0,i])[0]
    y = dft(diseased_r[1,i])[0]
    z = dft(diseased_r[2,i])[0]
    
    spec_roll_r[0,i] = spectral_rolloff(x)
    spec_roll_r[1,i] = spectral_rolloff(y)
    spec_roll_r[2,i] = spectral_rolloff(z)

### Spectral Centroid:

In [15]:
centr_s = np.ndarray((3,diseased_s.shape[1]))
for i in range(diseased_s.shape[1]):
    x = dft(diseased_s[0,i])[0]
    y = dft(diseased_s[1,i])[0]
    z = dft(diseased_s[2,i])[0]
                                
    centr_s[0,i] = spectral_centroid(x)
    centr_s[1,i] = spectral_centroid(y)
    centr_s[2,i] = spectral_centroid(z)
    
                                

In [16]:
centr_r = np.ndarray((3,diseased_r.shape[1]))
for i in range(diseased_s.shape[1]):
    x = dft(diseased_r[0,i])[0]
    y = dft(diseased_r[1,i])[0]
    z = dft(diseased_r[2,i])[0]
                                
    centr_r[0,i] = spectral_centroid(x)
    centr_r[1,i] = spectral_centroid(y)
    centr_r[2,i] = spectral_centroid(z)
    

### Spectral Spread

In [17]:
spec_spread_s = np.ndarray((3,diseased_s.shape[1]))
for i in range(diseased_s.shape[1]):
    x = dft(diseased_s[0,i])[0]
    y = dft(diseased_s[1,i])[0]
    z = dft(diseased_s[2,i])[0]
                                
    spec_spread_s[0,i] = spectral_spread(x)
    spec_spread_s[1,i] = spectral_spread(y)
    spec_spread_s[2,i] = spectral_spread(z)
    
                                

In [18]:
spec_spread_r = np.ndarray((3,diseased_r.shape[1]))
for i in range(diseased_r.shape[1]):
    x = dft(diseased_r[0,i])[0]
    y = dft(diseased_r[1,i])[0]
    z = dft(diseased_r[2,i])[0]
                                
    spec_spread_r[0,i] = spectral_spread(x)
    spec_spread_r[1,i] = spectral_spread(y)
    spec_spread_r[2,i] = spectral_spread(z)
    

### Create DataFrame

In [19]:
import pandas as pd

In [20]:
entropy_s.shape

(3, 1521)

### for S:


In [21]:
entropy_s = pd.DataFrame(entropy_s.T, columns =["E x", "E y", "E z"])
spec_entropy_s = pd.DataFrame(spec_ent_s.T, columns =["SE x", "SE y", "SE z"])
specroll_s = pd.DataFrame(spec_roll_s.T, columns =["SR x", "SR y", "SR z"])
spec_centr_s = pd.DataFrame(centr_s.T, columns =["SC x", "SC y", "SC z"])
spec_spread_s = pd.DataFrame(spec_spread_s.T, columns =["SS x", "SS y", "SS z"])

df_s = pd.concat([entropy_s, spec_entropy_s, specroll_s, spec_centr_s, spec_spread_s], axis =1)
df_s

,E x,E y,E z,SE x,SE y,SE z,SR x,SR y,SR z,SC x,SC y,SC z,SS x,SS y,SS z
0,3.321548,3.321678,3.321919,1.326060,0.388094,2.013945,0.206250,0.033594,0.295312,0.302684,0.259748,0.285944,0.281333,0.287911,0.247556
1,3.321668,3.321678,3.321920,1.749851,0.404518,2.075349,0.309375,0.028906,0.317188,0.328710,0.272052,0.299922,0.285380,0.290203,0.251753
2,3.321799,3.321751,3.321922,2.068534,0.489166,2.192129,0.396094,0.038281,0.346094,0.340882,0.300373,0.311415,0.285031,0.294217,0.251265
3,3.321825,3.321755,3.321922,2.105146,0.465596,2.137057,0.400000,0.037500,0.338281,0.338696,0.285988,0.313221,0.281644,0.289880,0.250696
4,3.321794,3.321758,3.321924,1.985826,0.487260,2.217650,0.381250,0.049219,0.345313,0.333969,0.281957,0.311462,0.285216,0.292110,0.254326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1516,3.315372,3.307817,3.321927,0.321362,0.710992,2.095766,0.000000,0.047656,0.523438,0.348181,0.369976,0.348897,0.322412,0.320644,0.297049
1517,3.316430,3.309940,3.321927,0.358661,0.829846,2.230188,0.001563,0.153125,0.570312,0.351130,0.374779,0.353584,0.323309,0.321474,0.294275
1518,3.317626,3.312692,3.321927,0.417753,0.913034,2.226873,0.017188,0.228125,0.571094,0.340209,0.365836,0.349573,0.324122,0.323764,0.296221
1519,3.317750,3.313168,3.321927,0.468278,1.003434,2.204400,0.025000,0.290625,0.576562,0.361177,0.372385,0.351756,0.321358,0.317678,0.299112


In [22]:
entropy_r = pd.DataFrame(entropy_r.T, columns =["E x", "E y", "E z"])
spec_entropy_r = pd.DataFrame(spec_ent_r.T, columns =["SE x", "SE y", "SE z"])
specroll_r = pd.DataFrame(spec_roll_r.T, columns =["SR x", "SR y", "SR z"])
spec_centr_r = pd.DataFrame(centr_r.T, columns =["SC x", "SC y", "SC z"])
spec_spread_r = pd.DataFrame(spec_spread_r.T, columns =["SS x", "SS y", "SS z"])

df_r = pd.concat([entropy_r, spec_entropy_r, specroll_r, spec_centr_r, spec_spread_r], axis =1)
df_r

,E x,E y,E z,SE x,SE y,SE z,SR x,SR y,SR z,SC x,SC y,SC z,SS x,SS y,SS z
0,3.321794,3.321779,3.321928,1.584578,0.551866,1.875754,0.217188,0.082031,2.882812e-01,0.278025,0.220101,2.776560e-01,0.268175,0.266221,0.254596
1,3.321919,3.321889,3.321928,1.740720,0.621221,2.084847,0.250000,0.089844,3.031250e-01,0.297544,0.250856,3.025639e-01,0.272042,0.272003,0.250905
2,3.321912,3.321909,3.321928,1.810876,0.651150,2.043323,0.289062,0.092969,2.976563e-01,0.286754,0.248992,2.921271e-01,0.265259,0.270524,0.251784
3,3.321833,3.321780,3.321928,1.736380,0.669318,2.042694,0.266406,0.098437,3.046875e-01,0.283527,0.237037,2.959837e-01,0.267794,0.269049,0.252682
4,3.321859,3.321870,3.321928,1.690290,0.652776,2.030060,0.241406,0.097656,2.929688e-01,0.283638,0.244276,2.967860e-01,0.266517,0.272164,0.251432
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012,3.321532,3.311345,3.321927,0.794899,0.347932,2.513784,0.572072,-0.136127,3.154749e+180,0.403641,0.292104,2.583923e+161,0.286042,0.264417,0.270685
2013,3.321484,3.304559,3.321925,2.311011,1.331310,2.568464,-0.124001,0.197016,2.212087e+214,0.120739,-0.966435,6.307379e-311,0.231898,0.215633,0.225259
2014,2.902293,3.149356,3.302838,0.658981,0.935960,0.541818,0.116211,-0.172135,1.428540e+248,-0.498068,-0.197797,2.583979e+161,0.209855,0.215413,0.211171
2015,2.715989,3.068933,3.211450,0.490014,0.609912,0.119040,0.257894,0.033885,5.134579e+199,0.070740,-0.412650,2.875173e+161,0.207599,0.222084,0.239102


In [23]:
df = pd.concat([df_s, df_r], axis = 0)
df

,E x,E y,E z,SE x,SE y,SE z,SR x,SR y,SR z,SC x,SC y,SC z,SS x,SS y,SS z
0,3.321548,3.321678,3.321919,1.326060,0.388094,2.013945,0.206250,0.033594,2.953125e-01,0.302684,0.259748,2.859439e-01,0.281333,0.287911,0.247556
1,3.321668,3.321678,3.321920,1.749851,0.404518,2.075349,0.309375,0.028906,3.171875e-01,0.328710,0.272052,2.999224e-01,0.285380,0.290203,0.251753
2,3.321799,3.321751,3.321922,2.068534,0.489166,2.192129,0.396094,0.038281,3.460937e-01,0.340882,0.300373,3.114147e-01,0.285031,0.294217,0.251265
3,3.321825,3.321755,3.321922,2.105146,0.465596,2.137057,0.400000,0.037500,3.382812e-01,0.338696,0.285988,3.132213e-01,0.281644,0.289880,0.250696
4,3.321794,3.321758,3.321924,1.985826,0.487260,2.217650,0.381250,0.049219,3.453125e-01,0.333969,0.281957,3.114619e-01,0.285216,0.292110,0.254326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012,3.321532,3.311345,3.321927,0.794899,0.347932,2.513784,0.572072,-0.136127,3.154749e+180,0.403641,0.292104,2.583923e+161,0.286042,0.264417,0.270685
2013,3.321484,3.304559,3.321925,2.311011,1.331310,2.568464,-0.124001,0.197016,2.212087e+214,0.120739,-0.966435,6.307379e-311,0.231898,0.215633,0.225259
2014,2.902293,3.149356,3.302838,0.658981,0.935960,0.541818,0.116211,-0.172135,1.428540e+248,-0.498068,-0.197797,2.583979e+161,0.209855,0.215413,0.211171
2015,2.715989,3.068933,3.211450,0.490014,0.609912,0.119040,0.257894,0.033885,5.134579e+199,0.070740,-0.412650,2.875173e+161,0.207599,0.222084,0.239102


In [24]:
df = df[np.random.permutation(df.columns)]
df

,SR x,E z,E x,E y,SR z,SS z,SE x,SE y,SE z,SC y,SS x,SC x,SS y,SC z,SR y
0,0.206250,3.321919,3.321548,3.321678,2.953125e-01,0.247556,1.326060,0.388094,2.013945,0.259748,0.281333,0.302684,0.287911,2.859439e-01,0.033594
1,0.309375,3.321920,3.321668,3.321678,3.171875e-01,0.251753,1.749851,0.404518,2.075349,0.272052,0.285380,0.328710,0.290203,2.999224e-01,0.028906
2,0.396094,3.321922,3.321799,3.321751,3.460937e-01,0.251265,2.068534,0.489166,2.192129,0.300373,0.285031,0.340882,0.294217,3.114147e-01,0.038281
3,0.400000,3.321922,3.321825,3.321755,3.382812e-01,0.250696,2.105146,0.465596,2.137057,0.285988,0.281644,0.338696,0.289880,3.132213e-01,0.037500
4,0.381250,3.321924,3.321794,3.321758,3.453125e-01,0.254326,1.985826,0.487260,2.217650,0.281957,0.285216,0.333969,0.292110,3.114619e-01,0.049219
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012,0.572072,3.321927,3.321532,3.311345,3.154749e+180,0.270685,0.794899,0.347932,2.513784,0.292104,0.286042,0.403641,0.264417,2.583923e+161,-0.136127
2013,-0.124001,3.321925,3.321484,3.304559,2.212087e+214,0.225259,2.311011,1.331310,2.568464,-0.966435,0.231898,0.120739,0.215633,6.307379e-311,0.197016
2014,0.116211,3.302838,2.902293,3.149356,1.428540e+248,0.211171,0.658981,0.935960,0.541818,-0.197797,0.209855,-0.498068,0.215413,2.583979e+161,-0.172135
2015,0.257894,3.211450,2.715989,3.068933,5.134579e+199,0.239102,0.490014,0.609912,0.119040,-0.412650,0.207599,0.070740,0.222084,2.875173e+161,0.033885


In [25]:
df.to_csv("/Users/ecem/Desktop/gyrocardiogram/s-vs-r/spectral_features_scg.csv")